In [1]:
import sys
import torch
import importlib

import silero_vad.silero_vad.utils as utils
import silero_vad.silero_vad.model as vad_model

In [46]:
importlib.reload(utils)
importlib.reload(vad_model)

from silero_vad.silero_vad.model import VoiceActivityDetection, VoiceActivityDetection_V5
from audiochains.streams import InputStream, StreamFromFile
from audiochains.writers import WriterInWAV
from audiochains.block_methods import RMSFromBytes, UnpackRawInInt16, UnpackRawInFloat32

In [17]:
from pathlib import Path

Path('./silero_vad/silero_vad/files/silero_vad.onnx').is_file()

True

In [47]:
model = VoiceActivityDetection(
    samplerate=16000,
    onnx_filepath='./silero_vad/silero_vad/files/silero_vad.onnx',
    # num_threads=1
)

In [48]:
model_v5 = VoiceActivityDetection_V5(
    samplerate=16000,
    onnx_filepath='./silero_vad/silero_vad/files/silero_vad_v5.onnx',
    # num_threads=1
)

In [50]:
# class UnpackRawInFloat64(BlockAudioMethod):
#     """
#     Unpacking raw audio data (sequences of bytes) in the numpy float32 array.
#     """

#     def __call__(self, in_data: bytes) -> np_float32_array:
#         max_int16_value = 2 ** 15
#         data = np.frombuffer(in_data, np.int16)
#         return data.astype(np.float32) / max_int16_value

In [51]:
torch.rand(1).to(torch.float64)

tensor([0.7193], dtype=torch.float64)

In [85]:
threshold = 0.5
blocksize = 512
sampwidth = 2


with StreamFromFile(
    filename='samples/учитель_дети_вырезка.wav',
    blocksize=blocksize
) as stream, WriterInWAV(
    file_name='samples_processed/processed.wav',
    framerate=stream.samplerate,
    channels=stream.channels,
    sampwidth=sampwidth
) as writer:
    stream.set_methods(
        UnpackRawInFloat32()
    )
    for _ in range(stream.get_iterations()):
        try:
            raw_data = stream.read(blocksize)
            confidence = model_v5(
                torch.from_numpy(stream.chain_of_methods(raw_data)), 16000
                ).item()
            if confidence >= threshold:
                writer.write(raw_data)
            else:
                writer.write(bytearray([0] * blocksize * 2))
        except Exception as err:
            print(f'VAD failed with exception: {err}')
            break

VAD failed with exception: Input audio chunk is too short


In [74]:
a = next(Path('samples/').iterdir())

In [75]:
a.stem

'папа'

In [93]:
threshold = 0.125
blocksize = 512
sampwidth = 2

for audio_file_path in Path('samples/').iterdir():
    print(f'{audio_file_path} in progres ...')
    with StreamFromFile(
        filename=str(audio_file_path),
        blocksize=blocksize
    ) as stream, WriterInWAV(
        file_name=f'samples_processed/{audio_file_path.stem}_processed.wav',
        framerate=stream.samplerate,
        channels=stream.channels,
        sampwidth=sampwidth
    ) as writer:
        stream.set_methods(
            UnpackRawInFloat32()
        )
        for _ in range(stream.get_iterations()):
            try:
                raw_data = stream.read(blocksize)
                confidence = model_v5(
                    torch.from_numpy(stream.chain_of_methods(raw_data)), 16000
                    ).item()
                if confidence >= threshold:
                    writer.write(raw_data)
                else:
                    writer.write(bytearray([0] * blocksize * 2))
            except Exception as err:
                print(f'VAD failed with exception: {err}')
                break

samples\будем_говорить_протяжно.wav in progres ...
VAD failed with exception: Input audio chunk is too short
samples\виноград_1.wav in progres ...
VAD failed with exception: Input audio chunk is too short
samples\виноград_2.wav in progres ...
VAD failed with exception: Input audio chunk is too short
samples\виноград_скажи.wav in progres ...
VAD failed with exception: Input audio chunk is too short
samples\говорят_о.wav in progres ...
VAD failed with exception: Provided number of samples is 1024 (Supported values: 256 for 8000 sample rate, 512 for 16000)
samples\дети_мычат.wav in progres ...
VAD failed with exception: Input audio chunk is too short
samples\загадкиwav.wav in progres ...
VAD failed with exception: Provided number of samples is 1024 (Supported values: 256 for 8000 sample rate, 512 for 16000)
samples\звук_с.wav in progres ...
VAD failed with exception: Input audio chunk is too short
samples\звуки.wav in progres ...
VAD failed with exception: Input audio chunk is too short
s

# Denoiser

In [96]:
import torchaudio
from IPython import display as disp

from denoiser import pretrained
from denoiser.dsp import convert_audio

In [108]:
wav[:, :1024].shape

torch.Size([1, 1024])

In [109]:
model = pretrained.dns64()
wav, sr = torchaudio.load('SA1.WAV.wav')
wav = wav[:, :1024]
wav = convert_audio(wav, sr, model.sample_rate, model.chin)
with torch.no_grad():
    denoised = model(wav[None])[0]
disp.display(disp.Audio(wav.data.cpu().numpy(), rate=model.sample_rate))
disp.display(disp.Audio(denoised.data.cpu().numpy(), rate=model.sample_rate))

In [41]:
import numpy as np
import wave

file = wave.open('SA1.WAV.wav')

In [45]:
arr = np.frombuffer(file.readframes(1024), np.int16).astype(np.float32) / 2**15
arr.shape

(1024,)

In [49]:
model(torch.from_numpy(arr), 16000)

tensor([[0.0259, 0.0278]])

In [55]:
from silero_vad.utils_vad import init_jit_model

model_jit = init_jit_model(
    model_path='silero_vad/files/silero_vad.jit'
)

model_jit(torch.from_numpy(arr), 16000).item()

0.024627722799777985